In [0]:
# Importação de bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
# carregando os dados
vendas_bronze = spark.read.format("delta").load("/Volumes/workspace/default/dadosvendas/bronze/vendas")
vendas_silver = spark.read.format("delta").load("/Volumes/workspace/default/dadosvendas/silver/vendas_enriquecidas")
vendas_gold = spark.read.format("parquet").load("/Volumes/workspace/default/dadosvendas/gold/vendas_consolidadas")

In [0]:
# criação das tabelas no catálogo

spark.sql("CREATE TABLE bronze.vendas USING DELTA LOCATION '/Volumes/workspace/default/dadosvendas/bronze/vendas'")
spark.sql("CREATE TABLE silver.vendas_enriquecidas USING DELTA LOCATION '/Volumes/workspace/default/dadosvendas/silver/vendas_enriquecidas'")

In [0]:
# VACUUM para limpar versões antigas dos nossos dados

spark.sql("VACUUM bronze.vendas RETAIN 168 HOURS")  # INDISPONÍVEL NO FREE EDITION
spark.sql("VACUUM silver.vendas_enriquecidas RETAIN 168 HOURS")  # INDISPONÍVEL NO FREE EDITION

In [0]:
# OPTIMIZE para compactar arquivos pequenos em maiores, melhorand performance de leitura

spark.sql("OPTIMIZE bronze.vendas")  # INDISPONÍVEL NO FREE EDITION
spark.sql("OPTIMIZE silver.vendas_enriquecidas")  # INDISPONÍVEL NO FREE EDITION

In [0]:
# Z-ORDER para otimização de consultas, clusteriza dados de colunas frequentes em filtros, acelerando queries com where em colunas específicas
spark.sql("OPTIMIZE silver.vendas_enriquecidas ZORDER BY (data_venda, id_vendedor)")  # INDISPONÍVEL NO FREE EDITION

In [0]:
# CACHE para armazenar em RAM tabelas frequentes, deixando bem mais rapidos nosso acesso nas consultas
spark.sql("CACHE TABLE silver.vendas_enriquecidas")  # INDISPONÍVEL NO FREE EDITION
spark.sql("CACHE TABLE gold.kpi_vendedores")  # INDISPONÍVEL NO FREE EDITION
